In [1]:
import ipyrad.analysis as ipa
import ipcoal
import toytree
import toyplot
import pandas as pd
from ipyrad.analysis.baba21 import Drawing

In [2]:
import random

def generate_tests_from_names(sources, targets_raw, outgroup, no_repeat=True):
    
    if type(sources) is not list:
        sources = [sources]
    
    if type(outgroup) is not list:
        outgroup = [outgroup]
    
    #Declare empty result
    tests =[]
    
    for source in sources:
        #If in sources are the outgroup skip it
        if source in outgroup:
            continue
        
        
        #New unlinked targets list every loop
        targets = targets_raw.copy()
            
        #Remove source and outgroups in case they are repeated in the targets list
        if source in targets:
            targets.remove(source)
        for i in outgroup:
            if i in targets:
                targets.remove(i) 

        #declare empty lists 
        included = []
        

        #Iterate over all targets
        for i, _ in enumerate(targets):
            #empty temp p variables
            p1 = [] 
            p2 = []

           #only do following if target was not already used (not in included)
            if targets[i] not in included:
                #use the first target as p1
                p1 = [targets[i]]
                #mark it is used appending to the list
                if no_repeat: included.append(targets[i])
                #avoid out of boundary errors
                if i < len(targets)-1:
                    #use the next target as p2
                    if targets[i+1] not in included:
                        p2 = [targets[i+1]]
                        #mask it as used
                        if no_repeat: included.append(targets[i+1])
                else:
                    #in case it is the last element (for odd number of targets, use a random one (but the current) as p2)
                    targets.remove(targets[i])
                    p2 =  [random.choice(targets)]

                #append test to the return list
                tests.append({'p1': p1, 'p2': p2, 'p3': [source], 'p4': outgroup})
                
    return tests #return list of dictionaries with every test

In [3]:
def get_significant_donee(df_result, test):
    if df_result["D"].values[0] > 0:
            significative_donee = test["p2"]
    elif df_result["D"].values[0] < 0:
        significative_donee = test["p1"]
    return significative_donee

In [ ]:
## Test abbas in old fashion way to calculate the number of positives.

##['70', '5', '18', [('r7', 'r15'), ('r7', 'r17'), ('r17', 'r6'), ('r18', 'r5')]]
import os
arr = os.listdir("/home/carlos/AutoABBA/scene_2/")

tips=20
tree = toytree.rtree.baltree(ntips=tips, treeheight=10e6)

final_result = []

for hdf5 in arr: 
    
#     trueSource, trueTarget = hdf5.split("_")[2].split(".")[0].split("to")
    trues = [i.split("to") for i in hdf5.split("_")[2].split(".")[0].split("and")]
    test_id = hdf5.split("_")[1]
    
    print (test_id)
    
    baba = ipa.baba21(f"/home/carlos/AutoABBA/scene_2/{hdf5}")
    source = tree.get_tip_labels()
    targets = tree.get_tip_labels()
    outgroup = "r19"

    tests = generate_tests_from_names(source, targets, outgroup, no_repeat=False)
    
    result = []
    for i in tests:
        try:
            df = baba.run_test(i, nboots=100, quiet=True)
        except:
            continue
        
        if df["Z"].values[0] > 2.5:
            pair = (*i["p3"], *get_significant_donee(df, i))
            if pair not in result:
                result.append(pair)



    
#     
#     print([test_id, trueSource, trueTarget, result])
#     final_result.append([test_id, trueSource, trueTarget, result])
#     print(result)
#     with open("fulldepur.txt", "a") as f:
#         f.write(str([test_id, trueSource, trueTarget, result])+ "\n")
    file = open("pureabbababa_s2.txt", "a")
    file.write(str([test_id, trues, result])+ "\n")
    file.close()

35


In [ ]:
print("t")